In [1]:
import tweepy
import csv
import itertools
import pandas as pd
import time
import sys

In [2]:
#Twitter API credentials
consumer_key = "BmvoUUcOUXhPxRR8uRC2TgKoW"
consumer_secret = "bNV6inRgeUSSVerytnnnTPveW8iM9GM0dwryZyiUKmYy436D1I"
access_key = "2969993776-b9Ui7fVJjW7gYId2C0kSGo5mN4ki93HSGEn6jx0"
access_secret = "N5ER33zjeIqfl5918MWTHLWbZzuBGfGL0FeSfNGvSsrvZ"

In [3]:
OAUTH_KEYS = {'consumer_key':consumer_key, 'consumer_secret':consumer_secret, 'access_token_key':access_key, 'access_token_secret':access_secret}
auth = tweepy.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [22]:
# Querying #confinement in Paris 10km radius with date limit and items() limit

search = tweepy.Cursor(api.search, q='#confinement',geocode="48.85717,2.34293,10km",until='2020-03-26').items(1)

In [6]:
# Create lists for each field desired from the tweets.
sn = []
text = []
timestamp =[]
for tweet in search:
    #print tweet.user.screen_name, tweet.created_at, tweet.text
    timestamp.append(tweet.created_at)
    sn.append(tweet.user.screen_name)
    text.append(tweet.text)

In [7]:
# Convert lists to dataframe
df2603 = pd.DataFrame()
df2603['timestamp'] = timestamp
df2603['sn'] = sn
df2603['text'] = text
df2603['timestamp'] = pd.to_datetime(df1703['timestamp'])

In [ ]:
# Subset for the dates required. Can select a specific date or time to examine.
#import time
#df = df[(pd.to_datetime("2015-12-14 20:00:00", format='%Y-%m-%d %H:%M:%S') < df['timestamp']) & (df['timestamp'] < pd.to_datetime("2015-12-14 21:00:00", format='%Y-%m-%d %H:%M:%S'))]

In [75]:
# Test df from previous query for creating getFriendship() function

df = pd.read_csv('corotweets.csv')

In [76]:
# Reduce 

df = df[:100]

In [77]:
df.head()

,timestamp,sn,text
0,2020-04-01 13:53:46,ErmannoKilgore,RT @MaxLandra: #Coronavirus\n#COVID19 \nIN SIL...
1,2020-04-01 13:53:46,SamsonHuang4,".\nFeel so blue today\n.\nHong Kong\nJan, 2020..."
2,2020-04-01 13:53:46,MrMatteoM,RT @stevesilberman: We're all reading a ton of...
3,2020-04-01 13:53:45,chidi_eboh,RT @Faithplantnig: Working diligently to bring...
4,2020-04-01 13:53:45,ShelleyBean01,RT @BeachMilk: A new rebellious trend has emer...


In [23]:
# Test show_friend method from tweepy

status=api.show_friendship(source_screen_name='ErmannoKilgore', target_screen_name='SamsonHuang4')
print(status[0].following, status[1].following)

False False


In [78]:
# Get all unique accounts and generate all possible pairs

allNames = list(df['sn'].unique())
allPairs = list(itertools.combinations(allNames, 2))

In [79]:
len(allPairs)

4950

# Not finished

def getFriendships(df):
    allNames = list(df['sn'].unique())
    allPairs = list(itertools.combinations(allNames, 2))
    a_fol_b_list = []
    b_fol_a_list = []
    for a,b in allPairs:
        status=api.show_friendship(source_screen_name=str(a), target_screen_name=str(b))
        a_fol_b=status[0].following
        b_fol_a=status[1].following
        a_fol_b_list.append(a_fol_b)
        b_fol_a_list.append(b_fol_a)
    

In [ ]:
a_fol_b_list = []
b_fol_a_list = []
for a,b in allPairs:
    status=api.show_friendship(source_screen_name=str(a), target_screen_name=str(b))
    a_fol_b=status[0].following
    b_fol_a=status[1].following
    a_fol_b_list.append(a_fol_b)
    b_fol_a_list.append(b_fol_a)

Rate limit reached. Sleeping for: 587


In [73]:
follows = pd.DataFrame()
follows['A_B'] = allPairs
follows['A_fol_B'] = a_fol_b_list
follows['B_fol_A'] = b_fol_a_list

In [74]:
follows

,A_B,A_fol_B,B_fol_A
0,"(ErmannoKilgore, SamsonHuang4)",False,False
1,"(ErmannoKilgore, MrMatteoM)",False,False
2,"(ErmannoKilgore, chidi_eboh)",False,False
3,"(ErmannoKilgore, ShelleyBean01)",False,False
4,"(SamsonHuang4, MrMatteoM)",False,False
5,"(SamsonHuang4, chidi_eboh)",False,False
6,"(SamsonHuang4, ShelleyBean01)",False,False
7,"(MrMatteoM, chidi_eboh)",False,False
8,"(MrMatteoM, ShelleyBean01)",False,False
9,"(chidi_eboh, ShelleyBean01)",False,False


In [ ]:
# Convert lists to dataframe
df2603 = pd.DataFrame()
df2603['timestamp'] = timestamp
df2603['sn'] = sn
df2603['text'] = text

In [ ]:
# Prepare for date filtering. Adding an EST time column since chat hosted by people in that time zone.
df2603['timestamp'] = pd.to_datetime(df2603['timestamp'])

In [ ]:
# Write out Tweets in case they are needed later.
df2603.to_csv('corotweets.csv',index = False,encoding='utf-8')

In [ ]:
# Create a list of the unique usernames in order to see which users we need to retrieve friends for.
allNames = list(df['sn'].unique())
print('Number of distinct usernames:')
print(len(allNames))

In [ ]:
# Initialize dataframe of users that will hold the edge relationships
dfUsers = pd.DataFrame()
dfUsers['userFromName'] =[]
dfUsers['userFromId'] =[]
dfUsers['userToId'] = []
count = 0

In [ ]:
# Again, to limit the number of calls to Twitter API, just do lookups on followers that connect to those in our user group.
# We are not interested in "friends" that are not part of this community.
fromId = dfUsers['userFromId'].unique()
dfChat = dfUsers[dfUsers['userToId'].apply(lambda x: x in fromId)]

In [ ]:
nameCount = len(allNames)
# The choice to retrieve friends (who the user is following) rather than followers is intentional.
# Either would work. However, many Twitter users follow fewer users than are following them, especially the most popular accounts. 
# This reduces the number of very large calls to Twitter API, which seemed to cause problems.
for name in allNames:
    # Build list of friends    
    currentFriends = []
    for page in tweepy.Cursor(api.friends_ids, screen_name=name).pages():
        currentFriends.extend(page)
    currentId = api.get_user(screen_name=name).id
    currentId = [currentId] * len(currentFriends)
    currentName = [name] * len(currentFriends)   
    dfTemp = pd.DataFrame()
    dfTemp['userFromName'] = currentName
    dfTemp['userFromId'] = currentId
    dfTemp['userToId'] = currentFriends
    dfUsers = pd.concat([dfUsers,dfTemp])
    time.sleep(5) # avoids hitting Twitter rate limit
    # Progress bar to track approximate progress
    count +=1
    per = round(count*100.0/nameCount,1)
    sys.stdout.write("\rTwitter call %s%% complete." % per)
    sys.stdout.flush()

In [ ]:
import plotly as py
from plotly.graph_objs import *

import networkx as nx
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Again, to limit the number of calls to Twitter API, just do lookups on followers that connect to those in our user group.
# We are not interested in "friends" that are not part of this community.
fromId = dfUsers['userFromId'].unique()
dfChat = dfUsers[dfUsers['userToId'].apply(lambda x: x in fromId)]

In [ ]:
# No more Twitter API lookups are necessary. Create a lookup table that we will use to get the verify the userToName
dfLookup = dfChat[['userFromName','userFromId']]
dfLookup = dfLookup.drop_duplicates()
dfLookup.columns = ['userToName','userToId']
dfCommunity = dfUsers.merge(dfLookup, on='userToId')

In [ ]:
dfCommunity.to_csv('dfCommunity.csv',index = False,encoding='utf-8')

In [ ]:
df = pd.read_csv('dfCommunity.csv')

In [ ]:
# Convert user ID from float to integer.
df.userFromId=df.userFromId.apply(lambda x: int(x))
df.userToId=df.userToId.apply(lambda x: int(x))

In [ ]:
G = nx.DiGraph()
#G = nx.Graph()
G.add_nodes_from(df['userFromId'])
#G.add_edges_from(zip(df['userFromId'],df['userToId']))

In [ ]:
temp = zip(df['userFromId'],df['userToId'])
G.add_edges_from(temp)

In [ ]:
# Give nodes their Usernames
dfLookup = df[['userFromName','userFromId']].drop_duplicates()

In [ ]:
dfLookup.head()
for userId in dfLookup['userFromId']:
    temp = dfLookup['userFromName'][df['userFromId']==userId]
    G.node[userId]['userName'] = temp.values[0]

In [ ]:
dfCommunity

In [ ]:
nx.draw(G, pos=nx.spring_layout(G,k=.12),node_color='c',edge_color='k')